In [13]:
import os
import io
import json
import base64
import requests
from PIL import Image
import numpy as np


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory



from requests.structures import CaseInsensitiveDict



from webbrowser import get
#from aiohttp import request
import streamlit as st
import matplotlib.pyplot as plt
import requests
import os
import random
from PIL import Image
#from mushroom_learning.utils import Wiki_Api, get_wiki_image, pic_to_dict

In [4]:
mushrooms = ['Agaricus_arvensis', 'Agaricus_impudicus', 'Amanita_muscaria', 'Amanita_virosa', 'Armillaria_lutea', 'Auricularia_auricula_judae', 'Basidioradulum_radula', 'Boletus_edulis', 'Byssomerulius_corium','Cantharellus_cibarius','Cerioporus_squamosus','Cerioporus_varius','Coprinellus_micaceus','Cortinarius_elatior','Cortinarius_flexipes','Cortinarius_malicorius','Cortinarius_torvus','Cuphophyllus_virgineus','Cylindrobasidium_laeve','Fomes_fomentarius','Fomitopsis_pinicola','Ganoderma_applanatum','Ganoderma_pfeifferi','Hericium_erinaceus','Hypholoma_fasciculare', 'Leccinum_scabrum', 'Mycena_galericulata', 'Plicatura_crispa', 'Pluteus_cervinus', 'Psathyrella_candolleana', 'Russula_mariae', 'Trametes_versicolor', 'Tricholoma_scalpturatum', 'Xerocomellus_chrysenteron', 'Xerocomus_ferrugineus', 'Xylodon_paradoxus']
len(mushrooms)

36

## Confirm everything is where it should be, and is of the correct size etc.

In [5]:
!pwd
!ls -rtlh amanita.jpg
! ls -rtlh ../model_1_species_vgg16/
! ls -rtlh ../model_2_species_vgg16/
! ls -rtlh ../model_3_species_vgg16/

/home/danielsptanner/code/DSP-Tan/mushroom_learning/notebooks
-rwxr-xr-x 1 danielsptanner danielsptanner 85K Mar  9 11:35 amanita.jpg
total 1.2M
-rw-r--r-- 1 danielsptanner danielsptanner  85K Mar 11 10:40 keras_metadata.pb
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 11 10:40 saved_model.pb
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 11 10:40 models_model_1_species_vgg16
drwxr-xr-x 2 danielsptanner danielsptanner 4.0K Mar 11 10:40 variables
total 656K
-rw-r--r-- 1 danielsptanner danielsptanner  85K Mar 10 18:05 keras_metadata.pb
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 10 18:05 saved_model.pb
drwxr-xr-x 2 danielsptanner danielsptanner 4.0K Mar 10 18:16 variables
total 1.2M
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 10 15:36 models_model_3_species_vgg16
-rw-r--r-- 1 danielsptanner danielsptanner  85K Mar 10 16:18 keras_metadata.pb
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 10 16:18 saved_model.pb
drwxr-xr-x 2 danielsptanner danielsptanner 4.0

## Set paths

In [6]:
names1=['Agaricus_arvensis','Agaricus_impudicus','Amanita_muscaria','Amanita_virosa','Armillaria_lutea',
        'Auricularia_auricula_judae','Basidioradulum_radula','Boletus_edulis','Byssomerulius_corium','Cantharellus_cibarius',
        'Cerioporus_squamosus','Cerioporus_varius','Other']

names2=['Coprinellus_micaceus','Cortinarius_elatior','Cortinarius_flexipes','Cortinarius_malicorius','Cortinarius_torvus',
        'Cuphophyllus_virgineus','Cylindrobasidium_laeve','Fomes_fomentarius','Fomitopsis_pinicola','Ganoderma_applanatum',
        'Ganoderma_pfeifferi','Hericium_erinaceus','Other']

names3=['Hypholoma_fasciculare','Leccinum_scabrum','Mycena_galericulata','Other','Plicatura_crispa','Pluteus_cervinus',
        'Psathyrella_candolleana','Russula_mariae','Trametes_versicolor','Tricholoma_scalpturatum','Xerocomellus_chrysenteron',
        'Xerocomus_ferrugineus','Xylodon_paradoxus']

In [ ]:
names1_common= ['Horse mushroom', 'Agaricus_impudicus', 'Fly agaric', 'Destroying angel', 'Armillaria gallica', "Jew's ear"
                'Toothed crust fungus', 'Penny bun', 'Netted crust fungus -check-', 
                'Chanterelle', "Dryad's saddle", "Elegant polypore", "Other"  ]

In [7]:
img_path='amanita.jpg'
model1_path='../model_1_species_vgg16/'
model2_path='../model_2_species_vgg16/'
model3_path='../model_3_species_vgg16/'

## Load models

In [8]:
model1=keras.models.load_model(model1_path)
model2=keras.models.load_model(model2_path)
model3=keras.models.load_model(model3_path)

2022-03-11 14:14:38.556718: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-11 14:14:38.556801: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-11 14:14:38.556826: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-2): /proc/driver/nvidia/version does not exist
2022-03-11 14:14:38.557206: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load and preprocess image.

In [9]:
size=(224,224)
batch_size = 32

img = tf.keras.utils.load_img("amanita.jpg", target_size=size)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

## Do prediction, results to dictionary

In [ ]:
prediction1 = model1.predict(img_array)
prediction2 = model2.predict(img_array)
prediction3 = model3.predict(img_array)
results1={names1[i]:score for i,score in enumerate(prediction1[0])}
results2={names2[i]:score for i,score in enumerate(prediction1[0])}
results3={names2[i]:score for i,score in enumerate(prediction1[0])}

In [ ]:
eval(str(results1))

## Stella's function

In [ ]:
def prediction_to_streamlit(pred_1,pred_2,pred_3):
    mushrooms = ['Agaricus_arvensis', 'Agaricus_impudicus', 'Amanita_muscaria', 'Amanita_virosa', 'Armillaria_lutea', 'Auricularia_auricula_judae', 'Basidioradulum_radula', 'Boletus_edulis', 'Byssomerulius_corium','Cantharellus_cibarius','Cerioporus_squamosus','Cerioporus_varius','Coprinellus_micaceus','Cortinarius_elatior','Cortinarius_flexipes','Cortinarius_malicorius','Cortinarius_torvus','Cuphophyllus_virgineus','Cylindrobasidium_laeve','Fomes_fomentarius','Fomitopsis_pinicola','Ganoderma_applanatum','Ganoderma_pfeifferi','Hericium_erinaceus','Hypholoma_fasciculare', 'Leccinum_scabrum', 'Mycena_galericulata', 'Plicatura_crispa', 'Pluteus_cervinus', 'Psathyrella_candolleana', 'Russula_mariae', 'Trametes_versicolor', 'Tricholoma_scalpturatum', 'Xerocomellus_chrysenteron', 'Xerocomus_ferrugineus', 'Xylodon_paradoxus']
    all_predictions = np.concatenate([pred_1,pred_2,pred_3],axis=1)
    max_value = np.argmax(all_predictions)
    predicted_mushroom = mushrooms[max_value]
    return max_value, predicted_mushroom

## Image preprocessing necessary for API

### This is what we  pass to the API

In [ ]:
# Set up image Jerome's way.
with open(img_path, "rb") as f:
    im_bytes = f.read()
im_b64 = base64.b64encode(im_bytes).decode("utf8")
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
payload = json.dumps({"image": im_b64})

In [ ]:
print(type(payload))
print(type(im_b64))
print(payload[0:20])
print(im_b64[0:10])

### This payload is a b64_utf8 string, with some headers

### This is what the API does to it to pass to model

#### Request with async function and await

In [ ]:
@app.post("/image")
async def process_image(request: Request):
    request_body = await request.json()

#### Gets the body image, which is a b64 string with utf8 encoding 

In [ ]:
image_b64_utf8 = request_body["image"]

#### Finally makes this b64_utf8 string suitable for passing to model

In [ ]:
def new_preproc(image_b64_utf8):
    size=(224,224)
    
    img_bytes = base64.b64decode(image_b64_utf8.encode('utf8'))
    # # convert bytes data to PIL Image object
    img = Image.open(io.BytesIO(img_bytes))
    img_arr = np.asarray(img)
    img_rs=tf.image.resize(img_arr,size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    img_exp = tf.expand_dims(img_rs, 0)
    return img_exp

In [ ]:
img_exp = new_preproc(im_b64)

In [ ]:
img_exp.shape

In [ ]:
model1.predict(img_exp)

In [ ]:
model1.predict(img_array)

In [17]:
! ls

01_DinaharDataset_primary.ipynb    laura_modeling_species.ipynb
02_UCIDataset.ipynb		   laura_poison_vgg19.ipynb
36_species_model_vgg16.ipynb	   laura_test_poison_model.ipynb
API_notebook.ipynb		   models_model_poison_simple_0.68
DataSet_Merging_Cleaning.ipynb	   models_model_species_simple_0.815
Minimal_Species_models_load.ipynb  new_local_query.py
SpeciesDatatSetExploration.ipynb   new_new_remote_query.py
Untitled.ipynb			   new_query_api.py
amanita.jpg			   new_remote_query.py
fart.jpg			   output.txt
laura_model_poison.ipynb	   query_api.py


In [18]:
image = Image.open('amanita.jpg')
buffer = io.BytesIO()
image.save(buffer, format='JPEG', quality=75)
im_bytes = buffer.getbuffer()
# with open(image, "rb") as file:
#     im_bytes = file.read()
image = base64.b64encode(im_bytes).decode("utf8")
headers = {'Content-type': 'application/json' , 'Accept' : 'text/plain'}
payload = json.dumps({"image" : image})

In [54]:
# Call speci²aes 1
url = "https://mushroom-docker-lpuaioudtq-ew.a.run.app/species1"
predict1 = requests.post(url , headers = headers , data = payload).json()
# This is a dictionary but we had to output as a string, so use eval
# to make it a dictionary again.
result1=eval(predict1)
print(type(result1))
result1

<class 'dict'>


{'Agaricus_arvensis': 1.299503e-07,
 'Agaricus_impudicus': 2.2494623e-06,
 'Amanita_muscaria': 0.9996468,
 'Amanita_virosa': 6.4207725e-08,
 'Armillaria_lutea': 3.9720445e-09,
 'Auricularia_auricula_judae': 2.4016526e-08,
 'Basidioradulum_radula': 4.1295083e-09,
 'Boletus_edulis': 0.00029080213,
 'Byssomerulius_corium': 8.519751e-11,
 'Cantharellus_cibarius': 5.9597376e-05,
 'Cerioporus_squamosus': 3.3394784e-10,
 'Cerioporus_varius': 4.4261683e-09,
 'Other': 3.844719e-07}

In [25]:
max_em=0
for key, value in result1.items():
    if value > max_em:
        if key=='Other':
            continue
        max_res=(key,value)
max_res        

('Cerioporus_varius', 4.4261683e-09)

In [45]:
def get_max_mush(dict_res):
    max_prob=0
    for key, value in dict_res.items():
        if key=='Other':
            continue
        if value > max_prob:
            max_res=(key,value)
            max_prob=value
    return max_res

In [46]:
res_1_max=get_max_mush(result1)

In [30]:
url = "https://mushroom-docker-lpuaioudtq-ew.a.run.app/species2"
predict2 = requests.post(url , headers = headers , data = payload).json()
# This is a dictionary but we had to output as a string, so use eval
# to make it a dictionary again.
result2=eval(predict2)
print(type(result2))
result2

<class 'dict'>


{'Coprinellus_micaceus': 0.007371621,
 'Cortinarius_elatior': 0.4490879,
 'Cortinarius_flexipes': 0.0055479864,
 'Cortinarius_malicorius': 0.015106743,
 'Cortinarius_torvus': 0.009534175,
 'Cuphophyllus_virgineus': 0.07830286,
 'Cylindrobasidium_laeve': 0.00638089,
 'Fomes_fomentarius': 0.00574864,
 'Fomitopsis_pinicola': 0.06738396,
 'Ganoderma_applanatum': 0.02719464,
 'Ganoderma_pfeifferi': 0.026327958,
 'Hericium_erinaceus': 0.00044051325,
 'Other': 0.30157214}

In [47]:
res_2_max=get_max_mush(result2)

In [34]:
url = "https://mushroom-docker-lpuaioudtq-ew.a.run.app/species3"
predict3 = requests.post(url , headers = headers , data = payload).json()
# This is a dictionary but we had to output as a string, so use eval
# to make it a dictionary again.
result3=eval(predict3)
print(type(result3))
result3

<class 'dict'>


{'Hypholoma_fasciculare': 0.2527563,
 'Leccinum_scabrum': 0.00024495702,
 'Mycena_galericulata': 0.0049598734,
 'Other': 0.18292418,
 'Plicatura_crispa': 0.004082506,
 'Pluteus_cervinus': 0.012862915,
 'Psathyrella_candolleana': 0.0027951954,
 'Russula_mariae': 0.19056284,
 'Trametes_versicolor': 0.020243945,
 'Tricholoma_scalpturatum': 0.03118043,
 'Xerocomellus_chrysenteron': 0.1622364,
 'Xerocomus_ferrugineus': 0.1351459,
 'Xylodon_paradoxus': 4.5929173e-06}

In [48]:
res_3_max=get_max_mush(result3)
print(res_3_max)

('Hypholoma_fasciculare', 0.2527563)


In [53]:
#get_max_mush(res_1_max,res_2_max,res_3_max)
dict(res_1_max)

ValueError: dictionary update sequence element #0 has length 16; 2 is required

In [49]:
get_max_mush({res_1_max[0]:res_1_max[1], res_2_max[0]:res_2_max[1], res_3_max[0]:res_3_max[1]})

('Amanita_muscaria', 0.9996468)

In [43]:
{res_1_max[0]:res_1_max[1], res_2_max[0]:res_2_max[1], res_3_max[0]:res_3_max[1]}

{'Cerioporus_varius': 4.4261683e-09,
 'Hericium_erinaceus': 0.00044051325,
 'Xylodon_paradoxus': 4.5929173e-06}